In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn import svm
import tensorflow as tf
#tf.enable_eager_execution()
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import itertools
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
data=pd.read_csv('chembl_24_bbb2.csv')
data.dropna(axis=1,how='all',inplace=True)
data.drop_duplicates(inplace=True)

In [7]:
data_f=pd.read_csv('chembl_24_BBB_full (2).csv')
#data_f.dropna(axis=1,how='all',inplace=True)
#data_f.drop_duplicates(inplace=True)
data_f[data_f['bbb_classification'].isna()==False]

,molregno,pref_name,chembl_id,max_phase,therapeutic_flag,dosed_ingredient,structure_type,chebi_par_id,molecule_type,first_approval,oral,parenteral,topical,black_box_warning,natural_product,first_in_class,chirality,prodrug,inorganic_flag,usan_year,availability_type,usan_stem,polymer_flag,usan_substem,usan_stem_definition,indication_class,withdrawn_flag,withdrawn_year,withdrawn_country,withdrawn_reason,withdrawn_class,molregno-2,mw_freebase,alogp,hba,hbd,psa,rtb,ro3_pass,num_ro5_violations,acd_most_apka,acd_most_bpka,acd_logp,acd_logd,molecular_species,full_mwt,aromatic_rings,heavy_atoms,qed_weighted,mw_monoisotopic,full_molformula,hba_lipinski,hbd_lipinski,num_lipinski_ro5_violations,molregno-3,molfile,standard_inchi,standard_inchi_key,canonical_smiles,compounds,bbb_value,bbb_classification,smiles,Oral bioavailability in monkey,"Dissociation constant, pKa of the compound",Partition coefficient (logP),ASTRAZENECA: Solubility in pH7.4
82,543,AMPHETAMINE,CHEMBL405,4,1,1,MOL,2679.0,Small molecule,1955.0,1,0,0,1,0,0,0,0,0,NaN,1,NaN,0,NaN,NaN,Stimulant (central),0,NaN,NaN,NaN,NaN,543,135.21,1.58,1.0,1.0,26.02,2.0,Y,0.0,NaN,9.94,1.79,-0.65,BASE,135.21,1.0,10.0,0.65,135.1048,C9H13N,1.0,2.0,NaN,543,\n SciTegic12231509382D\n\n 10 10 0 0 0 0...,"InChI=1S/C9H13N/c1-8(10)7-9-5-3-2-4-6-9/h2-6,8...",KWTSXDURSIMDCE-UHFFFAOYSA-N,CC(N)Cc1ccccc1,dataset3_comp58,0.930,0,CC(N)Cc1ccccc1,NaN,NaN,1.63 -,NaN
98,1354,BETAXOLOL,CHEMBL423,4,1,1,MOL,3082.0,Small molecule,1985.0,1,0,1,0,0,0,0,0,0,1983.0,1,-olol,0,-olol,beta-blockers (propranolol type),Antihypertensive; Anti-Anginal,0,NaN,NaN,NaN,NaN,1354,307.43,2.39,4.0,2.0,50.72,11.0,N,0.0,13.89,9.43,2.53,0.43,BASE,307.43,1.0,22.0,0.62,307.2147,C18H29NO3,4.0,2.0,NaN,1354,\n SciTegic12091219312D\n\n 22 23 0 0 0 0...,InChI=1S/C18H29NO3/c1-14(2)19-11-17(20)13-22-1...,NWIUTZDMDHAVTP-UHFFFAOYSA-N,CC(C)NCC(O)COc1ccc(CCOCC2CC2)cc1,dataset3_comp39,0.390,0,CC(C)NCC(O)COc1ccc(CCOCC2CC2)cc1,NaN,NaN,NaN,NaN
134,238429,ENFLURANE,CHEMBL1257,4,1,1,MOL,4792.0,Small molecule,1972.0,0,0,1,0,0,0,0,0,0,1970.0,1,-flurane,0,-flurane,general inhalation anesthetics (halogenated al...,Anesthetic (inhalation),0,NaN,NaN,NaN,NaN,238429,184.49,2.35,1.0,0.0,9.23,3.0,Y,0.0,NaN,NaN,2.16,2.16,NaN,184.49,0.0,10.0,0.48,183.9714,C3H2ClF5O,1.0,0.0,NaN,238429,"\n CDK 10/7/09,10:42\n\n 10 9 0 0 0 0...","InChI=1S/C3H2ClF5O/c4-1(5)3(8,9)10-2(6)7/h1-2H",JPGQOUSTVILISH-UHFFFAOYSA-N,FC(F)OC(F)(F)C(F)Cl,dataset11_comp45,0.240,-,FC(F)OC(F)(F)C(F)Cl,NaN,NaN,NaN,NaN
135,238429,ENFLURANE,CHEMBL1257,4,1,1,MOL,4792.0,Small molecule,1972.0,0,0,1,0,0,0,0,0,0,1970.0,1,-flurane,0,-flurane,general inhalation anesthetics (halogenated al...,Anesthetic (inhalation),0,NaN,NaN,NaN,NaN,238429,184.49,2.35,1.0,0.0,9.23,3.0,Y,0.0,NaN,NaN,2.16,2.16,NaN,184.49,0.0,10.0,0.48,183.9714,C3H2ClF5O,1.0,0.0,NaN,238429,"\n CDK 10/7/09,10:42\n\n 10 9 0 0 0 0...","InChI=1S/C3H2ClF5O/c4-1(5)3(8,9)10-2(6)7/h1-2H",JPGQOUSTVILISH-UHFFFAOYSA-N,FC(F)OC(F)(F)C(F)Cl,dataset4_CHEMBL1257,0.240,-,FC(F)OC(F)(F)C(F)Cl,NaN,NaN,NaN,NaN
136,238429,ENFLURANE,CHEMBL1257,4,1,1,MOL,4792.0,Small molecule,1972.0,0,0,1,0,0,0,0,0,0,1970.0,1,-flurane,0,-flurane,general inhalation anesthetics (halogenated al...,Anesthetic (inhalation),0,NaN,NaN,NaN,NaN,238429,184.49,2.35,1.0,0.0,9.23,3.0,Y,0.0,NaN,NaN,2.16,2.16,NaN,184.49,0.0,10.0,0.48,183.9714,C3H2ClF5O,1.0,0.0,NaN,238429,"\n CDK 10/7/09,10:42\n\n 10 9 0 0 0 0...","InChI=1S/C3H2ClF5O/c4-1(5)3(8,9)10-2(6)7/h1-2H",JPGQOUSTVILISH-UHFFFAOYSA-N,FC(F)OC(F)(F)C(F)Cl,dataset3_comp66,0.220,-,FC(F)OC(F)(F)C(F)Cl,NaN,NaN,NaN,NaN
173,789,METOPROLOL,CHEMBL13,4,1,1,MOL,6904.0,Small molecule,1978.0,1,1,0,1,0,0,0,0,0,1976.0,1,-olol,0,-olol,beta-blockers (propranolol type),"Anti-Adrenergic (beta-receptor),Antihypertensi...",0,NaN,NaN,NaN,NaN,789,267.37,1.61,4.0,2.0,50.72,9.0,N,0.0,13.89,9.43,1.63,-0.47,BASE,267.37,1.0,19.0,0.71,267.1834,C15H25NO3,4.0,2.0,NaN,789,\n SciTegic01111613442D\n\n 19 19 0 0 1 0...,InChI=1S/C15H25NO3/c1-12(2)16-10-14(17)11-19-1...,IUBSYMUCCVWXPE-UH

In [ ]:
BBB=pd.read_csv('../../DrugDatabasesTools/BBB_Files/BBB2.csv')
BBB.drop(columns=['compounds','BBB_value','SMILES'],inplace=True)
BBB.drop_duplicates(inplace=True)
tt=BBB.canonical_smiles.value_counts()>1
#Nc1nnc(-c2cccc(Cl)c2Cl)c(N)n1 True
#CC(Cl)(Cl)Cl True
#CN(C)CCCN1c2ccccc2CCc2ccccc21 True
#O=C1CC(=O)N(c2ccccc2)c2cc(Cl)ccc2N1 True

In [ ]:
full_1=pd.merge(data, BBB, left_on='canonical_smiles', right_on='canonical_smiles')
full_1

In [ ]:
pubchem=pd.read_csv('../../DrugDatabasesTools/PubChem/pubChem_chembl_24_bbb2.csv')
full_2=pd.merge(full_1, pubchem, left_on='canonical_smiles', right_on='original_smiles')
full_2

In [ ]:
rdkit=pd.read_csv('../../DrugDatabasesTools/Rdkit/rdkit_chembl_24_bbb2.csv')
rdkit.drop_duplicates(inplace=True)
full=pd.merge(full_2, rdkit, left_on='canonical_smiles', right_on='smiles')
full.head();

In [ ]:
full['smiles_length'] = [len(i) for i in full['canonical_smiles']]
full['fullmolformula_length'] = [len(i) for i in full['full_molformula']]
full['smile_to_formula']=full['smiles_length']/full['fullmolformula_length']

def molecule_type_to_numeric(x):
    if x=='Small molecule':
        return 1
    else:
        return 0
full['molecule_type_nr'] = full['molecule_type'].apply(molecule_type_to_numeric)
full.drop(columns='molecule_type',inplace=True)

def y_n_to_nr(x):
    if x=='Y':
        return 1
    elif x=='N':
        return 0
    else:
        return np.nan
full['ro3_pass_nr'] = full['ro3_pass'].apply(y_n_to_nr)
full.drop(columns='ro3_pass',inplace=True)
def molecular_species_to_nr(x):
    if x=='NEUTRAL':
        return 1
    elif x=='BASE':
        return 2
    elif x=='ACID':
        return 3
    else:
        return np.nan
full['molecular_species_nr'] = full['molecular_species'].apply(molecular_species_to_nr)
full.drop(columns='molecular_species',inplace=True)
def BBB_classification_to_nr(x):
    if x=='-':
        return 1
    elif x=='0':
        return 0
    elif x=='+':
        return 2
    else:
        return np.nan
full['BBB_classification_nr'] = full['BBB_classification'].apply(BBB_classification_to_nr)
full.drop(columns='BBB_classification',inplace=True)

full.drop(columns=["IUPACName","InChI","InChIKey","CanonicalSMILES","original_smiles","MolecularFormula","smiles","molregno", "pref_name","chembl_id","chebi_par_id","standard_inchi_key","canonical_smiles",'usan_year',"structure_type",'usan_substem',"usan_stem_definition","withdrawn_country","withdrawn_reason","molregno-2",
                  "molregno-3","molfile","standard_inchi","molfile","standard_inchi","Dissociation constant, pKa of the compound (units)","Partition coefficient (logP) (units)","full_molformula",'indication_class'],inplace=True)

category enconding, provided by [this](http://pbpython.com/categorical-encoding.html)  
cateogry way

In [ ]:
full["usan_stem"] = full["usan_stem"].astype('category')
full["usan_stem_cat"] = full["usan_stem"].cat.codes
full.drop(columns='usan_stem',inplace=True)

in the same link, get dummy way

In [ ]:
full=pd.get_dummies(full, columns=["withdrawn_class"])

In [209]:
cols = list(full)
nunique = full.apply(pd.Series.nunique)
cols_to_drop = nunique[nunique == 1].index
full.drop(cols_to_drop, axis=1,inplace=True)

scaling features and removing NA

In [210]:
features=full.columns.tolist()
features.remove('BBB_classification_nr')
full[features] = StandardScaler().fit_transform(full[features])

full.dropna(axis=1,how='any',inplace=True)
full.isna().sum()>0#178

max_phase                                                        False
therapeutic_flag                                                 False
dosed_ingredient                                                 False
oral                                                             False
parenteral                                                       False
topical                                                          False
black_box_warning                                                False
natural_product                                                  False
first_in_class                                                   False
chirality                                                        False
prodrug                                                          False
inorganic_flag                                                   False
availability_type                                                False
withdrawn_flag                                                   False
mw_fre

### Deep-learning with tf

In [ ]:
X=full.drop(columns='BBB_classification_nr')
y=full['BBB_classification_nr']

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X,y)

In [ ]:
training_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(X[features].values, tf.float32),
            tf.cast(y.values, tf.int32)
        )
    )
)

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(47,)),
    keras.layers.Dense(15, activation=tf.nn.relu),
    keras.layers.Dense(3, activation=tf.nn.softmax)
])
model.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=5)

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)

print('Test accuracy:', test_acc)

### Machine-learning --scikit-learn

In [211]:
def check_correlation(df,target,corr_cutoff):
    
    data_train=df.sample(frac=0.7,random_state=200)
    data_test=df.drop(data_train.index)

    data_x=df.drop(columns=[target])
    data_y=df[target]

    data_x_train=data_train.drop(columns=[target])
    data_y_train=data_train[target]

    data_x_test=data_test.drop(columns=[target])
    data_y_test=data_test[target]
    
    corr=data_train.corr()
    #fig,ax=plt.subplots(figsize=(8,6))
    #sns.heatmap(corr)
    features=''
    features_text=''
    if len(corr[target].where(lambda x : x.abs()>corr_cutoff).dropna())>1:
        features=corr[target].where(lambda x : x.abs()>corr_cutoff).dropna()
        features_text=features.index.str.cat(sep=', ')+'\n'
    else:
        features='1'
        features_text='None'
    print('The features correlated with target above the threshold %s are %s' %(corr_cutoff,features_text))
    return len(features)

check_correlation(full,'BBB_classification_nr',0.5)

The features correlated with target above the threshold 0.5 are None


1

In [212]:
result={}
from sklearn import utils
def xLinSVC(df,target,cv=10,C=1.0,multi_class ='ovr'):
    i=0
    f1=0
    score=0
    while i<cv:
        X=df.drop(columns=[target])
        y=df[target]
        X_train, X_test, y_train, y_test=train_test_split(X,y)
        utils.multiclass.type_of_target(y_train.astype('int'))
   
        clf = svm.LinearSVC(C=C,multi_class =multi_class )
        clf.fit(X_train, y_train) 
        score+=clf.score(X_test,y_test)
        f1+=f1_score(y_test,clf.predict(X_test),average='macro')
        i+=1
    return score/cv, f1/cv

for C in [5,10,11]:
        svr_score,f1_s=xLinSVC(full,'BBB_classification_nr',C=C)
        print(" Score is %s. and f1 is %s for %s" % ( svr_score, f1_s,C))
result['Linsvc_score']=svr_score
result['Linsvc_f1']=f1_s

C:\Users\joaof\Anaconda3\envs\py36\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\joaof\Anaconda3\envs\py36\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\joaof\Anaconda3\envs\py36\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\joaof\Anaconda3\envs\py36\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\joaof\Anaconda3\envs\py36\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear faile

 Score is 0.5192307692307693. and f1 is 0.35128138600007675 for 5


C:\Users\joaof\Anaconda3\envs\py36\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\joaof\Anaconda3\envs\py36\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\joaof\Anaconda3\envs\py36\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\joaof\Anaconda3\envs\py36\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\joaof\Anaconda3\envs\py36\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge

 Score is 0.573076923076923. and f1 is 0.37639999783399486 for 10


C:\Users\joaof\Anaconda3\envs\py36\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\joaof\Anaconda3\envs\py36\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\joaof\Anaconda3\envs\py36\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\joaof\Anaconda3\envs\py36\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\joaof\Anaconda3\envs\py36\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetric

 Score is 0.523076923076923. and f1 is 0.3465759953554071 for 11


In [213]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [214]:
def xSVC(df,target,cv=10):
    i=0
    f1=0
    score=0
    while i<cv:
        X=df.drop(columns=[target])
        y=df[target]
        X_train, X_test, y_train, y_test=train_test_split(X,y)
   
        clf= svm.SVC(gamma='scale', decision_function_shape='ovo')

        clf.fit(X_train, y_train) 
        score+=clf.score(X_test,y_test)
        f1+=f1_score(y_test,clf.predict(X_test),average='macro')
        i+=1
        cnf_matrix=confusion_matrix(y_test,clf.predict(X_test))
    return score/cv, f1/cv

svr_score,f1_s=xSVC(full,'BBB_classification_nr')
print(" Score is %s. and f1 is %s for %s" % ( svr_score, f1_s,C))
result['svc_score']=svr_score
result['svc_f1']=f1_s

 Score is 0.6807692307692308. and f1 is 0.46060779199099383 for 11


C:\Users\joaof\Anaconda3\envs\py36\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\joaof\Anaconda3\envs\py36\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\joaof\Anaconda3\envs\py36\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\joaof\Anaconda3\envs\py36\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\joaof\Anaco

In [215]:
def xLGR(df,target,cv=10):
    i=0
    f1=0
    score=0
    while i<cv:
        class_names = ['-','0','+']
        X=df.drop(columns=[target])
        y=df[target]
        X_train, X_test, y_train, y_test=train_test_split(X,y)
   
        lgr= LogisticRegression()

        lgr.fit(X_train, y_train) 
        score+=lgr.score(X_test,y_test)
        f1+=f1_score(y_test,lgr.predict(X_test),average='macro')
        i+=1
        cnf_matrix=confusion_matrix(y_test,lgr.predict(X_test))
        #plt.figure()
        #plot_confusion_matrix(cnf_matrix,classes=class_names)
    return score/cv, f1/cv

svr_score,f1_s=xLGR(full,'BBB_classification_nr')
print(" Score is %s. and f1 is %s" % ( svr_score, f1_s))
result["LogR_score"]=svr_score
result["LogR_f1"]=f1_s

 Score is 0.5884615384615385. and f1 is 0.3916921551620772


C:\Users\joaof\Anaconda3\envs\py36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\joaof\Anaconda3\envs\py36\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\joaof\Anaconda3\envs\py36\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\joaof\Anaconda3\envs\py36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\joaof\Anaconda3\envs\py36\lib\site-packages\sklearn\linear_mode

In [216]:
pd.DataFrame.from_dict(data=result,orient='index',columns=['result'])

,result
Linsvc_score,0.523077
Linsvc_f1,0.346576
svc_score,0.680769
svc_f1,0.460608
LogR_score,0.588462
LogR_f1,0.391692
